# Sandbox

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

## Models

In [ ]:
from transformers import CLIPProcessor, CLIPModel, CLIPVisionModelWithProjection

clip_model = CLIPVisionModelWithProjection.from_pretrained(
    pretrained_model_name_or_path='openai/clip-vit-large-patch14',
    ignore_mismatched_sizes=True
)
clip_model.to(device='cuda:0')

In [ ]:
from transformers import Dinov2Model

dv2_model = Dinov2Model.from_pretrained(
    pretrained_model_name_or_path='facebook/dinov2-base',
    ignore_mismatched_sizes=True
)
dv2_model.to(device='cuda:1')

In [ ]:
from PIL import Image
import src.data.transforms as dT
from src import utils
config = utils.get_config()
wandb_config = utils.load_config('clip.yml')
processor = dT.make_training_processor(config, wandb_config)
image = Image.open('/home/external-rosia/RosIA/reflection-connection/data/raw/train/Boring/abwao.png')
image = image.convert("RGB")
for _ in range(10):
    inputs = processor.preprocess_image([image])
    print(inputs.shape)

In [ ]:
outputs = clip_model(pixel_values=image.to(device='cuda:0'))
for k, v in outputs.items():
    display(k, v.shape)

In [ ]:
outputs = dv2_model(pixel_values=image.to(device='cuda:1'))
for k, v in outputs.items():
    display(k, v.shape)

In [ ]:
import torch
from torch import nn
from src.models.losses import make_triplet_criterion

input1 = torch.randn(1, 128)
anchors = torch.repeat_interleave(input1, repeats=2, dim=0)
# input2 = torch.randn(1, 128)
positives = torch.repeat_interleave(input1, repeats=2, dim=0)
negatives = torch.repeat_interleave(-input1, repeats=2, dim=0)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
display(1 - cos(anchors, positives))
display(1 - cos(anchors, negatives))
criterion = make_triplet_criterion({'criterion': 'TMWDL-Cosine'})
criterion(anchors, negatives, positives)

In [ ]:
from src.submissions import search
from PIL import Image
from src import utils
import torch

config = utils.get_config()
wandb_run = utils.get_run('2khs9u4f')
model = search.RefConInferenceModel(config, wandb_run, 0)
image = Image.open('/home/external-rosia/RosIA/reflection-connection/data/raw/train/Boring/abwao.png')
image = image.convert("RGB")
outputs = model(images=image)

del model
torch.cuda.empty_cache()

In [ ]:
from src.submissions import search
from PIL import Image
from src import utils
import torch

config = utils.get_config()
wandb_run = utils.get_run('2khs9u4f')

query_set = search.ImageSet(config, wandb_run, query=True, cuda_idx=0)
corpus_set = search.ImageSet(config, wandb_run, query=False, cuda_idx=0)
query_set.build_embeddings()
corpus_set.build_embeddings()
sbf = search.SearchBruteForce(corpus_set)
sbf.query(query_set)

In [ ]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests
import plotly.express as px

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
# model = AutoModel.from_pretrained('facebook/dinov2-base')
px.imshow(image).show()
inputs = processor(images=image, return_tensors="pt", do_normalize=False)
process_image = inputs['pixel_values'].squeeze().movedim(0, 2).numpy(force=True) * 255

px.imshow(process_image.tolist()).show()
# outputs = model(**inputs)
# last_hidden_states = outputs.last_hidden_state


In [ ]:
from typing import Sequence
import torch
import torchvision.transforms.functional as tvF
from torchvision import transforms
from PIL import Image
torch.tensor()
list_image = [Image.open(path)for path in ['/home/external-rosia/RosIA/reflection-connection/data/raw/train/Boring/ztqsz.png']*2]
tvF.to_tensor(list_image)

## FAISS

In [ ]:
import faiss
import numpy as np
import typing
arr = np.concatenate([np.array([]), ['a', 'b']])
arr[[1, 0]]

In [ ]:
import faiss
import numpy as np 
dataSetI = [[.1, .2, .3]]
dataSetI_I = [.4, .5, .6]
dataSetII = [.4, .5, .6]
# dataSetII = [.1, .2, .3]

x = np.array(dataSetI*3).astype(np.float32)
print(x)
q = np.array([dataSetII]).astype(np.float32)
print(q)
index = faiss.index_factory(3, "Flat", faiss.METRIC_INNER_PRODUCT)
faiss.normalize_L2(x)
index.add(x)
x2 = np.array([dataSetI_I]*3).astype(np.float32)
faiss.normalize_L2(x2)
index2 = faiss.index_factory(3, "Flat", faiss.METRIC_INNER_PRODUCT)
index2.add(x2)
index.merge_from(index2)
print(index.ntotal)

# index = faiss.index_cpu_to_all_gpus(index)
faiss.normalize_L2(q)
distance, return_index = index.search(q, 5)
print('Similarity by FAISS:{}'.format(distance))
print('Index by FAISS:{}'.format(return_index))


In [4]:
import torch
import torchvision.transforms.v2.functional as tvF

img = torch.arange(3*4*4).view(3,4,4)
img = tvF.to_image(img)
img = tvF.to_dtype_image(img, torch.float32, scale=True)
display(img)
tvF.to_dtype_image(img, torch.float32, scale=True)

tensor([[[0.0000e+00, 1.0842e-19, 2.1684e-19, 3.2526e-19],
         [4.3368e-19, 5.4210e-19, 6.5052e-19, 7.5894e-19],
         [8.6736e-19, 9.7578e-19, 1.0842e-18, 1.1926e-18],
         [1.3010e-18, 1.4095e-18, 1.5179e-18, 1.6263e-18]],

        [[1.7347e-18, 1.8431e-18, 1.9516e-18, 2.0600e-18],
         [2.1684e-18, 2.2768e-18, 2.3852e-18, 2.4937e-18],
         [2.6021e-18, 2.7105e-18, 2.8189e-18, 2.9273e-18],
         [3.0358e-18, 3.1442e-18, 3.2526e-18, 3.3610e-18]],

        [[3.4694e-18, 3.5779e-18, 3.6863e-18, 3.7947e-18],
         [3.9031e-18, 4.0115e-18, 4.1200e-18, 4.2284e-18],
         [4.3368e-18, 4.4452e-18, 4.5536e-18, 4.6621e-18],
         [4.7705e-18, 4.8789e-18, 4.9873e-18, 5.0958e-18]]])

tensor([[[0.0000e+00, 1.0842e-19, 2.1684e-19, 3.2526e-19],
         [4.3368e-19, 5.4210e-19, 6.5052e-19, 7.5894e-19],
         [8.6736e-19, 9.7578e-19, 1.0842e-18, 1.1926e-18],
         [1.3010e-18, 1.4095e-18, 1.5179e-18, 1.6263e-18]],

        [[1.7347e-18, 1.8431e-18, 1.9516e-18, 2.0600e-18],
         [2.1684e-18, 2.2768e-18, 2.3852e-18, 2.4937e-18],
         [2.6021e-18, 2.7105e-18, 2.8189e-18, 2.9273e-18],
         [3.0358e-18, 3.1442e-18, 3.2526e-18, 3.3610e-18]],

        [[3.4694e-18, 3.5779e-18, 3.6863e-18, 3.7947e-18],
         [3.9031e-18, 4.0115e-18, 4.1200e-18, 4.2284e-18],
         [4.3368e-18, 4.4452e-18, 4.5536e-18, 4.6621e-18],
         [4.7705e-18, 4.8789e-18, 4.9873e-18, 5.0958e-18]]])

In [3]:
import os

def get_class_for_image(image_name, class_folders):
    for folder in class_folders:
        if image_name in os.listdir(folder):
            return os.path.basename(folder)
    return None

def get_classes_for_images(image_names, parent_folder):
    class_folders = [os.path.join(parent_folder, folder) for folder in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, folder))]
    classes = [get_class_for_image(image_name, class_folders) for image_name in image_names]
    return classes

# Exemple d'utilisation
image_names = ["actea.png", "akytm.png", "abhkh.png"]  # Liste des noms d'images
parent_folder = "../data/raw/train"  # Chemin du dossier parent contenant les sous-dossiers/classes

classes = get_classes_for_images(image_names, parent_folder)
print(classes)


['Boring', 'Bright_Planar', 'Fault']
